In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "2g").\
        getOrCreate()

22/02/17 23:37:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df = spark.read.option("header", "true").csv("data/final_dataset.csv")
#df.printSchema()

In [3]:
from pyspark.sql.functions import split, explode, col, desc
from pyspark.ml import Pipeline 
from pyspark.ml.feature import VectorAssembler 
from pyspark.ml.feature import StringIndexer  
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

indexers = [
#StringIndexer(inputCol="Dst IP", outputCol = "dstip_index"),
StringIndexer(inputCol="Dst Port", outputCol = "dstport_index"),
StringIndexer(inputCol="FIN Flag Cnt", outputCol = "fin_index"),
StringIndexer(inputCol="SYN Flag Cnt", outputCol = "syn_index"),
StringIndexer(inputCol="URG Flag Cnt", outputCol = "urg_index"),
StringIndexer(inputCol="PSH Flag Cnt", outputCol = "cwe_index"),
StringIndexer(inputCol="ECE Flag Cnt", outputCol = "ece_index"),
#StringIndexer(inputCol="flow_dur", outputCol = "flowdur_index"),
StringIndexer(inputCol="Label", outputCol = "label")]

In [4]:
pipeline = Pipeline(stages=indexers)
df = df.select(#"Dst IP", 
               "Dst Port",
               "FIN Flag Cnt", 
               "SYN Flag Cnt", 
               "URG Flag Cnt",
               "PSH Flag Cnt",
               "ECE Flag Cnt",
#               "flow_dur",
               "Label"
              ).where(  (col("Dst Port") == "80") 
                      | (col("Dst Port") == "22")  
                      | (col("Dst Port") == "443") 
                      | (col("Dst Port") == "53")
                      | (col("Dst Port") == "25")
                      | (col("Dst Port") == "139")
                      | (col("Dst Port") == "445")
                      | (col("Dst Port") == "1433")
                      | (col("Dst Port") == "1434")
                      | (col("Dst Port") == "3389"))
#df = df.sample(fraction=0.3)
indexed_rows_df = pipeline.fit(df).transform(df)
#indexed_rows_df.show(5, False)

22/02/17 23:38:05 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
vectorAssembler = VectorAssembler(inputCols = ["dstport_index","fin_index", "syn_index", "urg_index", "cwe_index","ece_index"],outputCol = "features")
vindexed_rows_df = vectorAssembler.transform(indexed_rows_df)
#vindexed_rows_df.show(5, False)

In [6]:
splits = vindexed_rows_df.randomSplit([0.6,0.4], 42) 
# optional value 42 is seed for sampling 
train_df = splits[0] 
test_df = splits[1]

In [7]:
from pyspark.ml.classification import DecisionTreeClassifier

dtc = DecisionTreeClassifier(featuresCol="features", labelCol="label")
dtc = dtc.fit(train_df)

In [8]:
predictions_df = dtc.transform(test_df)
predictions_df.show(3)


+--------+------------+------------+------------+------------+------------+-----+-------------+---------+---------+---------+---------+---------+-------------+----------------+--------------------+----------+
|Dst Port|FIN Flag Cnt|SYN Flag Cnt|URG Flag Cnt|PSH Flag Cnt|ECE Flag Cnt|label|dstport_index|fin_index|syn_index|urg_index|cwe_index|ece_index|     features|   rawPrediction|         probability|prediction|
+--------+------------+------------+------------+------------+------------+-----+-------------+---------+---------+---------+---------+---------+-------------+----------------+--------------------+----------+
|     443|           0|           0|           0|           0|           0|  1.0|          2.0|      0.0|      0.0|      0.0|      0.0|      0.0|(6,[0],[2.0])|[2666910.0,72.0]|[0.99997300319237...|       0.0|
|     443|           0|           0|           0|           0|           0|  1.0|          2.0|      0.0|      0.0|      0.0|      0.0|      0.0|(6,[0],[2.0])|[2666

In [9]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", metricName="accuracy") 
nbaccuracy = evaluator.evaluate(predictions_df) 
print("Test accuracy = " + str(nbaccuracy))
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", metricName="f1")
nbf1 = evaluator.evaluate(predictions_df) 
print("F1 score = " + str(nbf1))

Test accuracy = 0.9377935817088255


F1 score = 0.9382120420699116
